In [1]:
#Imports
import sys
sys.path.append('../../python/')
import NGC5533_functions as nf

import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import lmfit as lm
import dataPython as dp

from datetime import datetime

In [2]:
data = dp.getXYdata_wXYerr('../data/100kpc_data.txt')
r_dat = np.asarray(data['xx'])
v_dat = np.asarray(data['yy'])
v_err0 = np.asarray(data['ex'])
v_err1 = np.asarray(data['ey'])

#change r_dat so it's strictly increasing
r_dat, v_dat, v_err0, v_err1 = (list(a) for a in zip(*sorted(zip(r_dat, v_dat, v_err0, v_err1))))
#converting v_err1 to an array
v_err1 = np.asarray(v_err1)
#Express as weights
weighdata = 1/v_err1

In [3]:
#keeping others but holding them constant
def f(r,M,h,d_rho00):
     return np.sqrt(nf.bh_v(r,M,load=True,path='../')**2 
                    + nf.h_v(r,load=True,path='../')**2 
                    + nf.b_v(r,load=True,path='../')**2 
                    + nf.d_v(r,h,d_rho00,pref=False,load=True,path='../')**2)
#Only Black Hole and Disk
def bh_d(r,M,h,d_rho00):
    return np.sqrt(nf.bh_v(r,M,load=True,path='../')**2 + nf.d_v(r,h,d_rho00,load=True,path='../')**2)

In [4]:
#Fit, Keeping rest constant

#Setup
l_mod = lm.Model(f)
l_params = l_mod.make_params()
#Black Hole
l_params.add('M', value=nf.Mbh_def, min=0) #Mass
#Disk
l_params.add('h', value=nf.h_c, min=0, max=1000)
l_params.add('d_rho00', value=nf.drho00_c, min=0)
#Do fit
l_fit = l_mod.fit(v_dat,l_params,r=r_dat,weights=weighdata)

In [ ]:
#Fit, BH and D only

#Setup
bhd_mod = lm.Model(bh_d)
bhd_params = bhd_mod.make_params()
#Black Hole
bhd_params.add('M', value=nf.Mbh_def, min=0) #Mass
#Disk
bhd_params.add('h', value=nf.h_c, min=0, max=1000)
bhd_params.add('d_rho00', value=nf.drho00_c, min=0)
#Do fit
bhd_fit = bhd_mod.fit(v_dat,bhd_params,r=r_dat,weights=weighdata)

../../python/NGC5533_functions.py:317: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return u*si.quad(d_innerfunc, 0, np.inf, args=(r,u,h,d_rho00))[0]
../../python/NGC5533_functions.py:321: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return si.quad(d_innerintegral, 0.1, 125, args=(r,h,d_rho00))[0]


In [ ]:
#Define curves to plot

l_dict = l_fit.best_values
l_M = l_dict['M']
l_h = l_dict['h']
l_d_rho00 = l_dict['d_rho00']
l_curve = f(r_dat,l_M,l_h,l_d_rho00)

bhd_dict = bhd_fit.best_values
bhd_M = bhd_dict['M']
bhd_h = bhd_dict['h']
bhd_d_rho00 = bhd_dict['d_rho00']

In [ ]:
fig = plt.figure(figsize=(9.0,8.0))

plt.xlim(0,125)

plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
plt.plot(r_dat,l_curve,'b-',label='Fit with All Components')
plt.plot(r_dat,bh_d(r_dat,l_M,l_h,l_d_rho00),'b--',label='Black Hole + Disk, from All Components')
plt.plot(r_dat,bh_d(r_dat,nf.Mbh_def,nf.h_c,nf.drho00_c),'g-.',label='Black Hole + Disk (Defaults)')
plt.plot(r_dat,bh_d(r_dat,bhd_M,bhd_h,bhd_d_rho00),label='Fit Black Hole + Disk Only')

plt.legend()
plt.show()

print('Fit information for all-component fit:')
l_fit

In [ ]:
fig = plt.figure(figsize=(9.0,8.0))

plt.xlim(0,25)
plt.ylim(0,400)

plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
plt.plot(r_dat,l_curve,'b-',label='Fit with All Components')
plt.plot(r_dat,bh_d(r_dat,l_M,l_h,l_d_rho00),'b--',label='Black Hole + Disk, from All Components')
plt.plot(r_dat,bh_d(r_dat,nf.Mbh_def,nf.h_c,nf.drho00_c),'g-.',label='Black Hole + Disk (Defaults)')
plt.plot(r_dat,bh_d(r_dat,bhd_M,bhd_h,bhd_d_rho00),label='Fit Black Hole + Disk Only')

plt.legend()
plt.show()

print('Fit information for two-component fit:')
bhd_fit

In [ ]:
print('Timestamp:')
print(datetime.now())